In [ ]:
"""
Universal backend test file.
Place this file in the Synesthesia/ folder and run:

    python test_backend.py

This tests processing for all emails in data/inbox.json:
- categorize_email
- action_item_extract
- autodraft_reply
- summary
"""

import os, sys, datetime, pprint
import json
import concurrent.futures
import time

# ---------------------------------------------------------
# FIX IMPORTS: add project root + backend to Python path
# ---------------------------------------------------------
try:
    # When running as a standard script
    ROOT = os.path.abspath(os.path.dirname(__file__))
except NameError:
    # When running in Jupyter/Interactive mode
    ROOT = os.path.abspath(os.getcwd())

BACKEND = os.path.join(ROOT, "backend")

sys.path.insert(0, ROOT)
sys.path.insert(0, BACKEND)

print("PROJECT ROOT:", ROOT)
print("BACKEND PATH:", BACKEND)

# ---------------------------------------------------------
# IMPORTS from your project
# ---------------------------------------------------------
from backend.agent.agent_orch import (
    categorize_email,
    action_item_extract,
    autodraft_reply,
    summary
)

pp = pprint.PrettyPrinter(indent=2)

# ---------------------------------------------------------
# Helper Functions
# ---------------------------------------------------------
def header(title: str):
    print("\n" + "=" * 60)
    print(title)
    print("=" * 60)

def process_email(email):
    """
    Runs the 4 core agent functions on a single email.
    Returns a dictionary of results.
    """
    try:
        body = email.get("body", "")
        subject = email.get("subject", "No Subject")
        email_id = email.get("id")

        # Run Agents (LLM calls)
        # We use json.loads where possible to format the output nicely, 
        # but fallback to raw string if the LLM output is messy (e.g. markdown blocks).
        
        raw_cat = categorize_email(body)
        try: category = json.loads(raw_cat)
        except: category = raw_cat

        raw_action = action_item_extract(body)
        try: actions = json.loads(raw_action)
        except: actions = raw_action

        raw_reply = autodraft_reply(body)
        try: reply = json.loads(raw_reply)
        except: reply = raw_reply

        summ = summary(body)

        return {
            "id": email_id,
            "subject": subject,
            "category": category,
            "actions": actions,
            "summary": summ.strip(),
            "reply": reply
        }
    except Exception as e:
        return {"error": f"Error processing {email.get('id')}: {str(e)}"}

# ---------------------------------------------------------
# MAIN EXECUTION
# ---------------------------------------------------------
def run_inbox_dry_run():
    inbox_path = os.path.join(ROOT, "data", "email_input.json")
    
    if not os.path.exists(inbox_path):
        print(f"\n[!] ERROR: Inbox file not found at: {inbox_path}")
        # Fallback check for notebook specific pathing
        fallback_path = os.path.join(ROOT, "Synesthesia", "data", "inbox.json")
        if os.path.exists(fallback_path):
             print(f"[*] Found at fallback path: {fallback_path}")
             inbox_path = fallback_path
        else:
             return

    print(f"\n[*] Loading inbox from: {inbox_path}")
    with open(inbox_path, "r", encoding="utf-8") as f:
        emails = json.load(f)

    print(f"[*] Found {len(emails)} emails. Starting parallel dry run (5 workers)...")
    start_time = time.time()

    # Run parallel processing
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        future_to_email = {executor.submit(process_email, email): email for email in emails}
        
        for i, future in enumerate(concurrent.futures.as_completed(future_to_email)):
            res = future.result()
            results.append(res)
            
            # Print results immediately as they complete
            print(f"\n--- [{i+1}/{len(emails)}] Processing ID: {res.get('id')} ---")
            
            if "error" in res:
                print(f"❌ {res['error']}")
            else:
                print(f"📧 Subject:  {res['subject']}")
                
                print(f"🏷️  Category:")
                pp.pprint(res['category'])
                
                print(f"⚡ Actions:")
                pp.pprint(res['actions'])
                
                print(f"📝 Summary:\n{res['summary']}")
                
                print(f"📨 Draft Reply:")
                pp.pprint(res['reply'])
            
            print("-" * 60)

    duration = time.time() - start_time
    print(f"\n[✓] Dry run complete. Processed {len(emails)} emails in {duration:.2f} seconds.")

# ---------------------------------------------------------
if __name__ == "__main__":
    run_inbox_dry_run()

In [ ]:
import sys
import os

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
import backend.utils.llm_cfg as llm_cfg
import backend.utils.sysprompts as sysprompts
from backend.utils import json_parser

def classify_intent(user_query: str):
    """
    Run ONLY the intent classifier.
    No email context, no categorization, no agents.
    """
    prompts = sysprompts.load_prompts()
    intent_prompt = prompts.get("sys_intent")

    full_prompt = f"""
{intent_prompt}

USER QUESTION:
{user_query}
"""

    raw_output = llm_cfg.run_llm(full_prompt)
    intent_json = json_parser.extract_json(raw_output)

    return intent_json.get("intent", "unknown")


if __name__ == "__main__":
    print("\n=== Intent Classification Tester ===")
    print("Type a query and press Enter.")
    print("Type 'exit' to quit.\n")

    while True:
        user_query = input("You: ")

        if user_query.lower().strip() in ["exit", "quit"]:
            break

        intent = classify_intent(user_query)
        print(f">>> Extracted Intent: {intent}\n")


In [ ]:
import sys
import os

# Ensure local imports work
sys.path.append(os.path.dirname(os.path.abspath(__file__)))

from backend.agent.agent_orch import orchestrator


# ---------------------------------------------------------
# Dummy email to test general queries
# ---------------------------------------------------------
DUMMY_EMAIL = """
Hi there,

Just checking in about the report you mentioned last week.
I didn’t see any attached file, so I’m not sure if something was missing
or if I misunderstood your previous message.

Let me know what you meant.

Thanks!
"""


def run_general_test():
    print("\n=== General Question Tester ===")
    print("Dummy email loaded.\n")
    print("EMAIL BODY:")
    print("--------------------------------")
    print(DUMMY_EMAIL)
    print("--------------------------------\n")

    while True:
        user_query = input("\nYour Question (or 'exit'): ")

        if user_query.lower().strip() in ["exit", "quit"]:
            print("\nExiting tester.\n")
            break

        print("\n>>> Running orchestrator...\n")

        result = orchestrator(
            email_body=DUMMY_EMAIL,
            user_question=user_query,
            use_rag=False,
            history=None
        )

        print("=== RESULT ===")
        print(f"Intent: {result.get('intent')}")
        print(f"Raw Output:\n{result.get('raw')}")
        print("==========================\n")


if __name__ == "__main__":
    run_general_test()


In [ ]:
import os
import sys

# Ensure this file can import agent_orch.py next to it
sys.path.append(os.path.dirname(os.path.abspath(__file__)))

from backend.agent.agent_orch import orchestrator

# -------------------------------
# Dummy Email for Testing
# -------------------------------
TEST_EMAIL = """
Hi Team,

This is a reminder that the final build review is scheduled for tomorrow at 3 PM. 
Please prepare your module summaries before the meeting.

Regards,
Alex
"""

# -------------------------------
# Test Queries
# -------------------------------
TEST_QUERIES = [
    "What category is this email?",
    "What do I need to do?",
    "Give me a summary.",
    "Draft a reply.",
    "Explain this email.",
    "Search for emails about build reviews."
]

def print_result(intent, result):
    print("\n==============================")
    print(f"USER INTENT → {intent}")
    print("==============================")

    print("RAW OUTPUT:")
    print(result.get("raw"))

    if result.get("json"):
        print("\nJSON PARSED:")
        print(result.get("json"))

    if result.get("results"):
        print("\nRAG RESULTS:")
        print(result.get("results"))

    print("==============================\n")


def run_tests():
    print("\n===== AGENT ORCHESTRATOR TEST HARNESS =====\n")
    print("Loaded dummy email:\n------------------------------------------")
    print(TEST_EMAIL)
    print("------------------------------------------\n")

    for q in TEST_QUERIES:
        print(f"\n>>> USER QUESTION: {q}")
        result = orchestrator(TEST_EMAIL, q)

        intent = result.get("intent")
        print_result(intent, result)

    print("\n===== TESTING COMPLETE =====\n")


if __name__ == "__main__":
    run_tests()


In [ ]:
"""
RAG Pipeline Test Script.
Place this in the Synesthesia/ folder and run:
    python test_rag.py

This script:
1. Builds the hybrid index (TF-IDF + ChromaDB) from your MongoDB emails.
2. Runs sample search queries to verify retrieval quality.
"""

import os
import sys
import pprint

# ---------------------------------------------------------
# PATH SETUP (Fixes ModuleNotFoundError)
# ---------------------------------------------------------
try:
    ROOT = os.path.abspath(os.path.dirname(__file__))
except NameError:
    ROOT = os.path.abspath(os.getcwd())

BACKEND = os.path.join(ROOT, "backend")
sys.path.insert(0, ROOT)
sys.path.insert(0, BACKEND)

print(f"[*] Project Root: {ROOT}")

# ---------------------------------------------------------
# IMPORTS
# ---------------------------------------------------------
try:
    from backend.rag.r import build_hybrid_index, hybrid_search
except ImportError as e:
    print(f"\n[!] Import Error: {e}")
    print("Ensure you have installed RAG dependencies:")
    print("pip install chromadb sentence-transformers scikit-learn joblib")
    sys.exit(1)

# ---------------------------------------------------------
# TEST FUNCTION
# ---------------------------------------------------------
def run_rag_test():
    print("\n" + "="*60)
    print("🚀 STARTING RAG ENGINE TEST")
    print("="*60)

    # 1. Build Index
    print("\n[*] Step 1: Building Hybrid Index...")
    print("    (This trains TF-IDF and embeds emails into ChromaDB)")
    
    try:
        success = build_hybrid_index()
        if not success:
            print("[!] Error: No emails found in MongoDB.")
            print("    Please run 'python backend/db/email_orch.py data/inbox.json' first.")
            return
        print("[✓] Indexing complete.")
    except Exception as e:
        print(f"[!] Critical Error building index: {e}")
        return

    # 2. Run Search Queries
    print("\n[*] Step 2: Testing Search Retrieval...")
    
    test_queries = [
        "security alert suspicious login",
        "deadline for compliance training",
        "lunch meeting plans",
        "project chimera code review"
    ]

    for query in test_queries:
        print(f"\n🔍 Query: '{query}'")
        try:
            # Retrieve top 3 results
            results = hybrid_search(query, top_k=3)
            
            if not results:
                print("   [!] No results returned.")
                continue

            for i, doc in enumerate(results):
                # Display metadata
                doc_id = doc.get('id', 'N/A')
                subject = doc.get('subject', 'No Subject')
                sender = doc.get('sender', 'Unknown')
                
                print(f"   {i+1}. [{doc_id}] {subject} (From: {sender})")
                
        except Exception as e:
            print(f"   [!] Search failed: {e}")

    print("\n" + "="*60)
    print("[✓] RAG Test Complete.")
    print("="*60)

if __name__ == "__main__":
    run_rag_test()

I'm sorry, I am an AI language model and do not have the ability to take photographs. However, if you would like me to provide you with a response related to photography or anything else you are interested in discussing, please let me know!
